使用keras搭建一个基于矩阵分解的推荐系统,而那篇文章所介绍的方法可能只是一个庞大推荐系统中的一小环节。而对于工业级别的推荐系统，面对极其庞大的产品种类数量，一步就输出符合用户心意的产品可能够呛，最好的方式应该是从巨大的产品类别之中粗筛出一些靠谱的待推荐产品，然后再从粗筛的产品中精挑细选出要推荐给用户的最终产品。

# 1 工业级别的推荐系统简介

工业级别的推荐系统的架构图如下图所示，大致分为两个阶段：

* **召回阶段**：也就是**粗筛阶段**，由于涉及到的产品数量巨大，大的公司都是千万级别，甚至上亿级别的产品数量，**此阶段的模型应该尽量简单，特征维度也尽量少，这样方便快速筛选出一些待推荐的产品。**

* **排序阶段**：即对上一阶段粗筛出来的待推荐产品进行**精挑细选**，此阶段**为了推荐出符合用户心意的产品，需要模型尽量的准确**。而且由于**粗筛阶段将数据量减少到几千，甚至几百级别，所以使用复杂模型，并且特征维度也可以尽量丰富，尽量多一些，这样训练出来的模型才能有较强的性能**。

![img](img/1.png)

而接下来我要介绍的FM（Factorization Machine）算法，不仅在召回阶段有用武之地，在排序阶段也是很拿得出手的推荐模型。

# 2 FM（Factorization Machine）算法简介

Factorization Machine的中文叫因子分解机，FM算法的最强特点就是**考虑到了特征的二阶组合——即特征两两组合形成一个新的特征。**

**在产品推荐，CTR预估等任务中，特征相互组合很可能会得到一个特别强的新特征。接下来我们从FM算法的公式来了解一下此算法的精髓：**

$$
y=w_{0}+\sum_{i=1}^{n} w_{i} x_{i}+\sum_{i=1}^{n} \sum_{i=j+1}^{n}<v_{i}, v_{j}>x_{i} x_{j}
$$

如果我们单看FM算法的前面一部分：

$$
y=w_{0}+\sum_{i=1}^{n} w_{i} x_{i}
$$

这不就是一个Logistics回归模型吗，确实没错，FM算法的前半部分就是Logistics回归，算法的后半部分才体现出FM的特征组合的思想：

![img](img/2.png)

$$
y=w_{0}+\sum_{i=1}^{n} w_{i} x_{i}+0.5 \sum_{i=1}^{k} \sum_{i=1}^{n}\left(\left(v_{i, f} x_{i}\right)^{2}-\sum_{i=1}^{n} v_{i, f}^{2} x_{i}^{2}\right)
$$

不要小看了公式的改写这一步，公式的改写这一过程会带来了算法时间复杂度的下降，加速算法的运行。接下来我们就尝试使用keras实现一下FM算法。

# 3 FM算法实战

In [1]:
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Input
from tensorflow.keras import Model
from tensorflow.keras import activations
from tensorflow.keras.optimizers import Adam

import tensorflow.keras.backend as K
from sklearn.datasets import load_breast_cancer

In [2]:
# FM层的定义，其中call函数中定义了FM的主要实现部分

class FM(Layer):
    def __init__(self, output_dim=30, activation="relu",**kwargs):
        self.output_dim = output_dim
        self.activate = activations.get(activation)
        super(FM, self).__init__(**kwargs)

    def build(self, input_shape):
        self.wight = self.add_weight(name='wight', 
                                      shape=(input_shape[1], self.output_dim),
                                      initializer='glorot_uniform',
                                      trainable=True)
        self.bias = self.add_weight(name='bias', 
                                      shape=(self.output_dim,),
                                      initializer='zeros',
                                      trainable=True)
        self.kernel = self.add_weight(name='kernel', 
                                      shape=(input_shape[1], self.output_dim),
                                      initializer='glorot_uniform',
                                      trainable=True)
        super(FM, self).build(input_shape)

    def call(self, x):
        feature =  K.dot(x,self.wight) + self.bias
        a = K.pow(K.dot(x,self.kernel), 2)
        b = K.dot(x, K.pow(self.kernel, 2))
        cross = K.mean(a-b, 1, keepdims=True)*0.5
        cross = K.repeat_elements(K.reshape(cross, (-1, 1)), self.output_dim, axis=-1)
        return self.activate(feature + cross)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

# 数据载入
载入sklearn中乳腺癌的分类任务数据。

In [3]:
data = load_breast_cancer()["data"]
target = load_breast_cancer()["target"]

# 模型构建
这里我采用了一层FM层，一层15个神经元的隐层构建了一个两层的网络模型，Loss 采用的是平方误差损失（mse），当然也可以采用交叉熵损失（cross entropy）。

In [5]:
K.clear_session()

inputs = Input(shape=(30,))
out = FM(20)(inputs)
out = Dense(15,activation="sigmoid")(inputs)
out = Dense(1,activation="sigmoid")(out)

model = Model(inputs=inputs, outputs=out)
model.compile(loss='mse',
              optimizer=Adam(0.0001),
              metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 30)]              0         
_________________________________________________________________
dense (Dense)                (None, 15)                465       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 16        
Total params: 481
Trainable params: 481
Non-trainable params: 0
_________________________________________________________________


# 模型训练
定义好batch_size 和训练轮数，就可以将模型跑起来了。

In [6]:
model.fit(data, target,
            batch_size=1,
            epochs=100,
            validation_split=0.2)

Train on 455 samples, validate on 114 samples
Epoch 1/100
455/455 [==============================] - 0s 1ms/sample - loss: 0.2441 - accuracy: 0.5912 - val_loss: 0.1956 - val_accuracy: 0.7719
Epoch 2/100
455/455 [==============================] - 0s 738us/sample - loss: 0.2427 - accuracy: 0.5912 - val_loss: 0.1921 - val_accuracy: 0.7719
Epoch 3/100
455/455 [==============================] - 0s 759us/sample - loss: 0.2343 - accuracy: 0.5912 - val_loss: 0.1848 - val_accuracy: 0.7719
Epoch 4/100
455/455 [==============================] - 0s 786us/sample - loss: 0.2301 - accuracy: 0.5912 - val_loss: 0.1846 - val_accuracy: 0.7719
Epoch 5/100
455/455 [==============================] - 0s 808us/sample - loss: 0.2277 - accuracy: 0.5912 - val_loss: 0.1838 - val_accuracy: 0.7719
Epoch 6/100
455/455 [==============================] - 0s 769us/sample - loss: 0.2252 - accuracy: 0.5912 - val_loss: 0.1837 - val_accuracy: 0.7719
Epoch 7/100
455/455 [==============================] - 0s 776us/sample - l

FM在应对特征丰富的推荐任务时有着很不错的效果。毕竟考虑到了特征之间的组合关系。